# Importing and preparing supermarkets data

## Libraries and settings

In [131]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: /Users/darioholenstein/Documents/zhaw/data-analytics/data_analytics/Week_02
supermarkets.json


## Importing data

In [132]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [133]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [134]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2 = df2[['brand', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,shop,city,street,housenumber,postcode
0,Spar,supermarket,NaN,NaN,NaN,NaN
1,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,Coop,supermarket,Uznach,NaN,NaN,8730
3,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Merge df1 and df2

In [135]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Count and identify the number of missing values (if any)

In [136]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type              0
id                0
lat               0
lon               0
brand          1065
shop              0
city           1777
street         1608
housenumber    1680
postcode       1709
dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
5,node,60271452,47.406671,9.305450,NaN,supermarket,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,supermarket,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,supermarket,NaN,NaN,NaN,NaN
13,node,95582038,47.050385,9.059214,NaN,supermarket,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,supermarket,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,supermarket,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN


## Count and identify duplicated values (if any)

In [137]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode


## Get data types of all variables

In [138]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type            object
id               int64
lat            float64
lon            float64
brand           object
shop            object
city            object
street          object
housenumber     object
postcode        object
dtype: object

### Save data to file

In [139]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Filter for Migros

In [140]:
df_filtered_migros_zurich = df.loc[(df['brand'] == 'Migros') & (df['city'] == 'Zürich')]
df_filtered_migros_zurich.head()

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
11,node,83330862,47.344749,8.529981,Migros,supermarket,Zürich,Etzelstrasse,3,8038
16,node,119249170,47.375255,8.536107,Migros,supermarket,Zürich,Löwenstrasse,31-35,8001
50,node,262400822,47.364072,8.530945,Migros,supermarket,Zürich,Tessinerplatz,10,8002
71,node,267346993,47.385598,8.531471,Migros,supermarket,Zürich,Limmatstrasse,152,8005


### Filter and count all Coop supermarkets in the cities of Zürich, Basel & Bern

In [141]:
df_filtered_coop_zurich = df.loc[(df['brand'] == 'Coop') & (df['city'] == 'Zürich')]
print('Coops in Zurich:', df_filtered_coop_zurich.shape[0])
df_filtered_coop_zurich.head(df_filtered_coop_zurich.shape[0])

Coops in Zurich: 37


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
9,node,79977755,47.340070,8.530546,Coop,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
59,node,265776668,47.376417,8.559594,Coop,supermarket,Zürich,Zürichbergstrasse,75,8044
63,node,266630559,47.377716,8.511219,Coop,supermarket,Zürich,Badenerstrasse,333,8003
70,node,267345511,47.385809,8.516574,Coop,supermarket,Zürich,Maagplatz,1,8005
72,node,267468996,47.364872,8.521006,Coop,supermarket,Zürich,Uetlibergstrasse,20,8045
75,node,268603429,47.367360,8.546174,Coop,supermarket,Zürich,Theaterstrasse,18,8001
81,node,270692983,47.357940,8.554646,Coop,supermarket,Zürich,Seefeldstrasse,123,8008
84,node,271028686,47.366773,8.548079,Coop,supermarket,Zürich,Stadelhoferstrasse,10,8001
86,node,271029581,47.364300,8.555129,Coop,supermarket,Zürich,Forchstrasse,4,8008


In [142]:
df_filtered_coop_basel = df.loc[(df['brand'] == 'Coop') & (df['city'] == 'Basel')]
print('Coops in Basel:', df_filtered_coop_basel.shape[0])
df_filtered_coop_basel.head(df_filtered_coop_basel.shape[0])

Coops in Basel: 10


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
686,node,707688653,47.569095,7.579643,Coop,supermarket,Basel,Elsässerstrasse,36,4056
1013,node,1322305936,47.545220,7.588710,Coop,supermarket,Basel,Güterstrasse,125,4053
1113,node,1513116290,47.569082,7.590574,Coop,supermarket,Basel,Klybeckstrasse,142,4057
1145,node,1598176926,47.543122,7.593269,Coop,supermarket,Basel,Güterstrasse,190,4053
1148,node,1598954903,47.555210,7.613846,Coop,supermarket,Basel,Farnsburgerstrasse,37,4052
1617,node,2663868342,47.570592,7.573368,Coop,supermarket,Basel,Vogesenplatz,10,4056
2132,node,4342407034,47.565114,7.617822,Coop,supermarket,Basel,Bäumlihofstrasse,132/134,4058
2185,node,4437700046,47.555654,7.590741,Coop,supermarket,Basel,Freie Strasse,75,4001
2928,node,8098492384,47.561065,7.591392,Coop,supermarket,Basel,Greifengasse,3,4058
2929,node,8098492385,47.576962,7.591138,Coop,supermarket,Basel,Inselstrasse,76,4057


### Filter supermarkets with available brand, city, house number and postcode. 

In [155]:
df_full_data = df.loc[(df['brand'].notna()) & (df['city'].notna()) & (df['housenumber'].notna()) & (df['postcode'].notna())]
df_full_data.head()

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406
8,node,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038


### Include opening hours as additional variable in the data frame.

In [146]:
df3 = pd.DataFrame.from_records(df1.tags)
df3 = df3[['opening_hours']]
df_with_hours = pd.merge(df, 
              df3,
              left_index=True, 
              right_index=True)
df_with_hours.head()

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730,NaN
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off


### Filter supermarkets with available opening hours.

In [157]:
df_has_hours = df_with_hours.loc[(df_with_hours['opening_hours'].notna())]
df_has_hours.head()

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [144]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.5.0
Datetime: 2024-09-25 13:26:55
Python Version: 3.11.5
-----------------------------------
